# Исследование надежности заемщиков


Задача:
На основе данных из банка определить клиент, который может уйти

## Откроем таблицу и изучим общую информацию о данных

In [ ]:
import pandas as pd

try:
    data = pd.read_csv('/datasets/data.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')

In [ ]:
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Подвывод: мы прочитали исходные данные, посмотрели на их структуру, тип и объём, что понадобится для дальнейшей работы.

## Предобработка данных

### Удаление пропусков

In [ ]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [ ]:
data[data['days_employed'].isna()]
#Одинаковое количество пропусков в обоих столбцах наводит на мысль а односторонней или взаимной зависимости пропусков.
#Также можно предположить что пропуски неслучайные, так как зависят от характера вопроса. Например, люди могут просто
#не вспомнить свой точный стаж и пропустить этот пункт совсем, могут не указать total_income так как боятся получить
#отказ кредита или им просто неприятно сообщать эту информацию.

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Среднее,1,женат / замужем,0,M,компаньон,0,NaN,сделка с автомобилем
21495,1,NaN,50,среднее,1,гражданский брак,1,F,сотрудник,0,NaN,свадьба
21497,0,NaN,48,ВЫСШЕЕ,0,женат / замужем,0,F,компаньон,0,NaN,строительство недвижимости
21502,1,NaN,42,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,строительство жилой недвижимости


In [ ]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

### Обработка аномальных значений

In [ ]:
data['days_employed'] = data['days_employed'].abs()

In [ ]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получатся аномально большие значения. Исправить такие значения сложно, поэтому оставьте их как есть. Тем более этот столбец не понадобится вам для исследования.

In [ ]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

In [ ]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

In [ ]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5])

### Удаление пропусков (продолжение)

In [ ]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

In [ ]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Изменение типов данных

In [ ]:
data['total_income'] = data['total_income'].astype(int)

### Обработка дубликатов

In [ ]:
data['education'] = data['education'].str.lower()

In [ ]:
data.duplicated().sum()

71

In [ ]:
data = data.drop_duplicates()

### Категоризация данных

In [ ]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [ ]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

In [ ]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

In [ ]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [ ]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

Подвывод: в этом разделе мы провели предобработку данных:
        
 - нашли и устранили дубликаты
 - устранили аномальные значения в колонке children
 - нашли и обработали пропуски на основе типа данных этой колонки и данных, от которых зависила колонка с пропусками
 - провели категоризацию на основе цели получения кредита и уровня дохода клиента

### Шаг 3. Исследуйте данные и ответьте на вопросы

#### 3.1 Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [ ]:
data[data['debt'] == 1]['debt'].sum()
#Всего есть 1732 клиентов с долгами

1732

In [ ]:
#Создаем сводную таблицу широкого формата, чтобы впоследствии сравнить колонки
debt_children_pivot = data.pivot_table(index='children', values='debt', aggfunc=['count', 'sum'])
debt_children_pivot['ratio'] = debt_children_pivot['sum']/debt_children_pivot['count']
#Чем меньше значение в колонке ratio, тем больше вероятность что клиент вернёт долг

In [ ]:
#Сверяем, не потеряли ли мы какие-нибудь данные по долгам
print(debt_children_pivot['sum'].sum())
#Количество долгов совпадает с изначальным количеством

debt    1732
dtype: int64


In [ ]:
debt_children_pivot

,count,sum,ratio
,debt,debt,
children,,,
0,14091,1063,0.075438
1,4808,444,0.092346
2,2052,194,0.094542
3,330,27,0.081818
4,41,4,0.097561
5,9,0,0.000000


In [ ]:
# Код ревьюера
data.pivot_table(index = 'children', values = 'debt', aggfunc = ['count', 'sum', 'mean'])

,count,sum,mean
,debt,debt,debt
children,,,
0,14091,1063,0.075438
1,4808,444,0.092346
2,2052,194,0.094542
3,330,27,0.081818
4,41,4,0.097561
5,9,0,0.000000


**Вывод:**
На основе полученных данных можно сопоставить только те категории граждан, по которым есть достаточно большое количество данных: клиенты с 0-2 детьми. Выяснилось, что клиенты без детей наиболее вероятно останутся без долгов, чем клиенты с 1 или 2 детьми. Те, в свою очередь, имеют примерно одинаковую вероятность неоплаты долга.

#### 3.2 Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [ ]:
#Делаем то же самое, что и в предыдущем пункте
debt_family_status_pivot = data.pivot_table(index='family_status', values='debt', aggfunc=['count', 'sum'])
debt_family_status_pivot['ratio'] = debt_family_status_pivot['sum']/debt_family_status_pivot['count']
debt_family_status_pivot

,count,sum,ratio
,debt,debt,
family_status,,,
Не женат / не замужем,2796,273,0.097639
в разводе,1189,84,0.070648
вдовец / вдова,951,63,0.066246
гражданский брак,4134,385,0.093130
женат / замужем,12261,927,0.075606


**Вывод:** Вдовцы/вдовы, клиенты в разводе, а также женатые/замужние клиенты с наибольшей вероятностью вернут долг.

#### 3.3 Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

##### Первый способ категоризации (ручная делёжка)

In [ ]:
#Разделим разницу максимального и минимального дохода на 4, таким образом получим 4 сегмента (квартиля/четверти)
income_segment = (data['total_income'].max() - data['total_income'].min()) / 4

In [ ]:
#Напишем функцию, которая будет возвращать категорию в зависимости от того, в какой квартиль попадает
#доход конкретого клиента
def income_group(row):
    try:
        if row <= data['total_income'].min() + income_segment:
            return 'низкий'
        elif row <= data['total_income'].min() + income_segment * 2:
            return 'ниже среднего'
        elif row <= data['total_income'].min() + income_segment * 3:
            return 'выше среднего'
        return 'высокий'
    except:
        pass

In [ ]:
#Создаём новый столбец с результатами функции
data['income_group'] = data['total_income'].apply(income_group)
data

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category,income_group
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью,низкий
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем,низкий
2,0,5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью,низкий
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования,низкий
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы,низкий
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем,B,операции с недвижимостью,низкий
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем,C,операции с автомобилем,низкий
21522,1,2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость,C,операции с недвижимостью,низкий
21523,3,3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля,B,операции с автомобилем,низкий


Сверка результатов:

In [ ]:
data[data['total_income']==21895]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category,income_group
14276,0,346602.453782,61,среднее,1,женат / замужем,0,F,пенсионер,0,21895,недвижимость,E,операции с недвижимостью,низкий


In [ ]:
data[data['income_group']=='ниже среднего']

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category,income_group
39,0,650.587796,31,высшее,0,гражданский брак,1,F,компаньон,0,754240,заняться образованием,B,получение образования,ниже среднего
68,2,2152.475526,46,высшее,0,женат / замужем,0,M,компаньон,0,592071,операции с коммерческой недвижимостью,B,операции с недвижимостью,ниже среднего
722,1,541.449660,32,высшее,0,женат / замужем,0,M,компаньон,0,672306,сделка с автомобилем,B,операции с автомобилем,ниже среднего
725,1,244.421041,56,среднее,1,в разводе,3,F,компаньон,0,596496,заняться высшим образованием,B,получение образования,ниже среднего
884,0,2308.271667,35,высшее,0,женат / замужем,0,M,госслужащий,0,673040,на покупку своего автомобиля,B,операции с автомобилем,ниже среднего
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20262,0,2456.551956,29,высшее,0,Не женат / не замужем,4,M,компаньон,0,661635,строительство жилой недвижимости,B,операции с недвижимостью,ниже среднего
20388,1,330469.273975,49,высшее,0,гражданский брак,1,M,пенсионер,0,624655,на проведение свадьбы,B,проведение свадьбы,ниже среднего
20663,0,1305.382866,42,высшее,0,женат / замужем,0,M,сотрудник,0,647456,операции с коммерческой недвижимостью,B,операции с недвижимостью,ниже среднего
21295,0,352.048655,51,высшее,0,Не женат / не замужем,4,F,сотрудник,0,816023,приобретение автомобиля,B,операции с автомобилем,ниже среднего


In [ ]:
data[data['income_group']=='выше среднего']

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category,income_group
2224,1,4614.851849,42,высшее,0,женат / замужем,0,M,сотрудник,0,1278622,покупка жилья,A,операции с недвижимостью,выше среднего
7447,0,8626.777234,39,среднее,1,в разводе,3,F,сотрудник,0,1240165,покупка недвижимости,A,операции с недвижимостью,выше среднего
11071,1,1851.200013,36,высшее,0,гражданский брак,1,F,сотрудник,0,1286280,покупка коммерческой недвижимости,A,операции с недвижимостью,выше среднего
15211,0,2009.086175,34,среднее,1,женат / замужем,0,F,сотрудник,0,1172459,получение образования,A,получение образования,выше среднего
15268,1,10207.448165,64,высшее,0,в разводе,3,M,компаньон,0,1350245,жилье,A,операции с недвижимостью,выше среднего
17503,0,2285.476482,43,среднее,1,женат / замужем,0,M,компаньон,0,1597613,операции с недвижимостью,A,операции с недвижимостью,выше среднего
18353,1,3173.282035,41,высшее,0,Не женат / не замужем,4,F,компаньон,0,1427934,автомобиль,A,операции с автомобилем,выше среднего
18368,1,333.935516,41,высшее,0,гражданский брак,1,M,компаньон,0,1551152,свадьба,A,проведение свадьбы,выше среднего
19813,0,2672.323106,50,высшее,0,женат / замужем,0,F,компаньон,0,1223042,покупка коммерческой недвижимости,A,операции с недвижимостью,выше среднего


In [ ]:
data[data['income_group']=='высокий']

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category,income_group
9169,1,5248.554336,35,среднее,1,гражданский брак,1,M,сотрудник,0,1726276,дополнительное образование,A,получение образования,высокий
12412,0,1477.438114,44,высшее,0,женат / замужем,0,M,компаньон,0,2265604,ремонт жилью,A,операции с недвижимостью,высокий
17178,0,5734.127087,42,высшее,0,гражданский брак,1,M,компаньон,0,1711309,сыграть свадьбу,A,проведение свадьбы,высокий
19606,1,2577.664662,39,высшее,0,женат / замужем,0,M,компаньон,1,2200852,строительство недвижимости,A,операции с недвижимостью,высокий
20809,0,4719.273476,61,среднее,1,Не женат / не замужем,4,F,сотрудник,0,1715018,покупка жилья для семьи,A,операции с недвижимостью,высокий


In [ ]:
#Создаём сводную таблицу, как и в предыщущих пунктах
debt_income_group_pivot = data.pivot_table(index='income_group', values='debt', aggfunc=['count', 'sum'])
debt_income_group_pivot['ratio'] = debt_income_group_pivot['sum']/debt_income_group_pivot['count']
debt_income_group_pivot

,count,sum,ratio
,debt,debt,
income_group,,,
высокий,5,1,0.20000
выше среднего,9,0,0.00000
ниже среднего,109,7,0.06422
низкий,21208,1724,0.08129


##### Второй способ категоризации

In [ ]:
#Сводная таблица по категориям дохода п.16
debt_total_income_group_pivot = data.pivot_table(index='total_income_category', values='debt', aggfunc=['count', 'sum'])
debt_total_income_group_pivot['ratio'] = debt_total_income_group_pivot['sum']/debt_total_income_group_pivot['count']
debt_total_income_group_pivot

,count,sum,ratio
,debt,debt,
total_income_category,,,
A,25,2,0.080000
B,5014,354,0.070602
C,15921,1353,0.084982
D,349,21,0.060172
E,22,2,0.090909


**Вывод:** Основываясь на категоризации из п.16 мы получаем только 2 категории с достаточным количеством данных для вывода: B и C. Клиент, принадлежащий категории B более вероятно оплатит по долгам, чем клиент из категории C.

##### Третий способ категоризировать доходы (qcut)

In [ ]:
data['third_income_cat'] = pd.qcut(x=data['total_income'], q=5, labels=['low', 'lower_avg', 'avg', 'higher_avg', 'high'])
data['third_income_cat']

0             high
1        lower_avg
2              avg
3             high
4              avg
           ...    
21520         high
21521          avg
21522          low
21523         high
21524          low
Name: third_income_cat, Length: 21331, dtype: category
Categories (5, object): ['low' < 'lower_avg' < 'avg' < 'higher_avg' < 'high']

In [ ]:
#Сводная таблица по категориям из qcut()
debt_third_income_cat_pivot = data.pivot_table(index='third_income_cat', values='debt', aggfunc=['count', 'sum'])
debt_third_income_cat_pivot['ratio'] = debt_third_income_cat_pivot['sum']/debt_third_income_cat_pivot['count']
debt_third_income_cat_pivot

,count,sum,ratio
,debt,debt,
third_income_cat,,,
low,4267,344,0.080619
lower_avg,4266,358,0.083919
avg,4266,373,0.087436
higher_avg,4266,358,0.083919
high,4266,299,0.070089


**Вывод**: после применения qcut() мы получили равные по размерам категории, которые можно сравнить (по сравнению с прошлыми, где можно было сравнить только 2 группы клиентов). Из таблицы видно, что клиенты с самым высоким уровнем дохода наиболее вероятно останутся без долгов, потом идут клиенты с самыми низкими доходами (они вернут долг с меньшей вероятностью), а вероятность оставшихся категорий одинаково мала. Результаты этой таблицы мы и будет использовать в общем выводе.

#### 3.4 Как разные цели кредита влияют на его возврат в срок?

In [ ]:
#В целях есть следующие категории:
data['purpose_category'].value_counts()

операции с недвижимостью    10751
операции с автомобилем       4279
получение образования        3988
проведение свадьбы           2313
Name: purpose_category, dtype: int64

In [ ]:
#Создадим сводную таблицу:
debt_purpose_debt_pivot = data.pivot_table(index='purpose_category', values='debt', aggfunc=['count', 'sum'])
debt_purpose_debt_pivot['ratio'] = debt_purpose_debt_pivot['sum']/debt_purpose_debt_pivot['count']
debt_purpose_debt_pivot

,count,sum,ratio
,debt,debt,
purpose_category,,,
операции с автомобилем,4279,400,0.093480
операции с недвижимостью,10751,780,0.072551
получение образования,3988,369,0.092528
проведение свадьбы,2313,183,0.079118


**Вывод:** Меньше всего долгов у категории граждан, берущих кредит на операции с недвижимостью, чуть больше долгов у тех, кто брал кредит для свадьбы. У категорий "автомобиль" и "образование" одинаково большое отношение неоплаченных долгов.

### Шаг 4: общий вывод.

Мы проанализировали личные данные клиентов и посмотрели, какие факторы влияют на вероятность, что конкретный клиент вернёт долг. Перед непосредственным анализом мы обработали пропущенные значения в данных, исправили аномалии и обработали дубликаты. Далее мы разделили всех клиентов на несколько групп в зависимости от их уровня дохода и цели взятия кредита.

В ходе непосредственного анализа выяснилось, что самыми надежными будут клиенты:
 - без детей (вернут долг с вероятностью 92,5%)
 - состоящие или состоявшие в браке (~93%)
 - относящиеся к категории с самыми высоким и самыми низкими доходами (93% и 92%)

Самыми ненадежными будут клиенты:
 - с 1-2 детьми (90%)
 - вне брака или в гражданском браке (90,3% и 90,7%)
 - со средним доходом (91,2%)

Для более точных результатов в будущем, стоит обратить внимание на природу пропущенных данных в колонках total_income и days_employed, так как данные в них могут не записываться из-за технической неисправности системы сбора данных. Возможно система не сохраняет данные, если они не соответствуют каким-то внутренним критериям или при отсутсвии ответа в одной из упомянутых колонок удаляет данные и в другой. Также возможной причиной может быть необязательность заполнения этих данных, чем злоупотребляют клиенты, не желая тратить время.